In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)

df = pd.DataFrame.from_records(shelter.read({}))

#df.drop(columns=['_id'],inplace=True)

print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         style={'height':'20%', 'width':'20%'})),
    html.Center(html.B(html.H3("Derek Bamford-SNHU CS340- Project 2"))),
    
    html.Hr(),
    
    html.Center(
        dcc.Dropdown(
        id='filter-type',
          options=[
            {'label':'Show All', 'value': 'All'},
            {'label':'Water Rescue', 'value': 'Water'},
            {'label':'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label':'Disaster Rescue or Individual Tracking', 'value':'Disaster'},
          ],
    
            value='All', #Sets the default dropdown to 'All'
            multi=False, #Does not allow for multiple selections
            clearable=False, #We do not want the dropdown to have no value
            style={'width': '400px', 'text-align' : 'center'}
        )  
    ),
    
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, 
                  "selectable": True} for i in df.columns],
        
        data=df.to_dict('records'),
        
        editable=True,#makes the tables editable
        filter_action="native", #allows for filtering
        sort_action="native", #allows for sorting
        sort_mode="multi", #allows to sort more than one field
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0], # Set to zero To prevent issues with not selecting an initial row
        page_action="native",
        page_current=0, #sets start page to the beginning
        page_size=10 #sets rows per page
                        ),
     html.Br(),
     html.Hr(),
    
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
            ])
                    ])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id', 'columns')],
              [Input('filter-type', 'value')]
            )
              
def update_dashboard(filter_type):
    
    #Filter for water rescue 
        if filter_type == 'Water':
            print('Water Selected')
            #Reference: Query commands taken from Week 7 quiz.
            df=pd.DataFrame.from_records(shelter.read({
                "animal_type":"Dog",
                "breed": {"$in": ["Chesapeake Bay Retriever","Labrador Retriever Mix","Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        #Filter for Mountain rescue
        elif filter_type == "Mountain":
            print('Mountian Selected')
            df=pd.DataFrame.from_records(shelter.read({
                "animal_type":"Dog", 
                "breed":{"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}   
            }))
        
        #Filter for Disaster Rescue
        elif filter_type == "Disaster":
            print('Disaster Selected')
            df=pd.DataFrame.from_records(shelter.read({
                "animal_type":"Dog", 
                "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
                "sex_upon_outcome": "Intact Male", 
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            })) 
    
        #Reset the filters
        elif filter_type == "All":
            print('Reset Selected')
            df=pd.DataFrame.from_records(shelter.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data,columns)
        
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)

def update_graphs(viewData):
    df = pd.DataFrame.from_records(viewData)
    print('Graph Activated')
    
    #reference https://plotly.com/python/pie-charts/#customizing-a-pie-chart-created-with-pxpie
    figure = px.pie(df, names='breed', title='Preferred Animals')
    figure.update_traces(textposition='inside')
    figure.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
    
    return dcc.Graph(figure=figure)
         


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else:
        row = index[0]

# Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, 
            children=[dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], 
                    children=[dl.Tooltip(dff.iloc[row,4]),
                    dl.Popup([html.H1("Animal Name"),html.P(dff.iloc[row,9])])
                            ])
                    ])
    ]

app.run_server(debug=True)

Connection Successful
Read Code Reached
10004
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', 'address'],
      dtype='object')
Dash app running on http://127.0.0.1:10398/
Reset Selected
Read Code Reached
Graph Activated
Water Selected
Read Code Reached
Graph Activated
Mountian Selected
Read Code Reached
Graph Activated
Disaster Selected
Read Code Reached
Graph Activated
